#필요한 패키지 설치

In [ ]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 1.3 MB/s 
     |████████████████████████████████| 5.8 MB 62.1 MB/s 
     |████████████████████████████████| 1.3 MB 51.0 MB/s 
     |████████████████████████████████| 182 kB 42.1 MB/s 
     |████████████████████████████████| 7.6 MB 46.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4f3ebe5a67181985706a6b543edaca27f8de9a3a8461f4c1a30b96807ba49364
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [ ]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/744 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/585 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

[[-0.37510476 -0.77338415  0.5927711  ...  0.57923514  0.3268347
  -0.65089625]
 [-0.09361722 -0.18191545 -0.19230829 ... -0.03165793  0.3041255
  -0.26793614]]


# 데이터셋 로드

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# AI 허브의 감성말뭉치 데이터 불러오기
train_data = pd.read_excel('/content/drive/MyDrive/NLP_Project/감성대화/Training/감성대화말뭉치(최종데이터)_Training.xlsx')
val_data = pd.read_excel('/content/drive/MyDrive/NLP_Project/감성대화/Validation/감성대화말뭉치(최종데이터)_Validation.xlsx')
all_df = pd.concat([train_data, val_data]).iloc[:, 1:]

In [ ]:
# 데이터에서 필요한 컬럼만 추출한 후 뒤에서 다른 데이터를 병합하기 위해 컬럼명 변경
all_df = all_df[['사람문장1','시스템응답1','감정_대분류']]
all_df.columns = ['Q','A','label']

In [ ]:
all_df

,Q,A,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2
...,...,...,...
5125,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,여유롭게 생활할 수 있어 좋으시겠어요.,2
5126,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,치료가 완료되지 않은 게 아닐까 싶으셔서 염려하고 계시는군요.,1
5127,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,소개팅에서 만나는 사람과 계속 잘 안 풀려서 많이 좌절하신 것 같아요.,1
5128,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,은행 대출이 안 돼서 마음이 괴로우시군요.,1


In [ ]:
# label은 감정대분류인데 크게 6가지 감정으로 분류되어있지만 오타로 인해 기쁨이 2개가 있다. 따로 라벨링
all_df['label'].unique()

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [ ]:
# 감정 -> 0 : 중립, 1 : 부정 , 2: 긍정
all_df['label'] = all_df['label'].map({'기쁨' : 2, '기쁨 ':2 ,'불안' : 1, '당황':1, '슬픔':1, '분노':1,'상처':1,'불안 ':1})

In [ ]:
all_df  # 라벨링된 데이터셋

,Q,A,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2
...,...,...,...
5125,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,여유롭게 생활할 수 있어 좋으시겠어요.,2
5126,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,치료가 완료되지 않은 게 아닐까 싶으셔서 염려하고 계시는군요.,1
5127,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,소개팅에서 만나는 사람과 계속 잘 안 풀려서 많이 좌절하신 것 같아요.,1
5128,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,은행 대출이 안 돼서 마음이 괴로우시군요.,1


In [ ]:
# 새로운 챗봇 데이터 로드
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
df2 = pd.read_csv('ChatBotData.csv')
df2

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [ ]:
# 말뭉치 데이터와 챗봇 데이터를 병합하여 데이터량을 46009 -> 57832로 늘렸다.
df = pd.concat([all_df,df2])
df.reset_index(drop=True, inplace=True)
df

,Q,A,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2
...,...,...,...
57827,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
57828,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
57829,흑기사 해주는 짝남.,설렜겠어요.,2
57830,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


# 전처리

In [ ]:
# 결측치 NaN 처리
df = df[~df['A'].isna()]
df.head()

,Q,A,label
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2


In [ ]:
# 0번 문장 인코딩해보기
df.loc[0, 'Q'] 

'아내가 드디어 출산하게 되어서 정말 신이 나.'

In [ ]:
# 인코딩 예시
model.encode(df.loc[0, 'Q'])

array([-4.58952308e-01, -1.65475816e-01, -1.16999483e+00, -5.23580968e-01,
        6.83550090e-02, -1.33704454e-01,  2.92058587e-01, -2.52802402e-01,
       -9.53771174e-02, -8.38729978e-01,  5.42452693e-01, -1.65980637e-01,
       -6.91887856e-01,  7.94256330e-01, -9.43957627e-01, -6.27517462e-01,
        4.37359095e-01,  6.75986707e-01, -5.40876925e-01, -5.64558923e-01,
       -2.99052242e-03,  6.71544522e-02, -1.08207650e-01, -2.11770646e-02,
        1.15095809e-01, -3.29753906e-01, -3.89415979e-01, -1.94628865e-01,
       -2.79100120e-01,  6.35069847e-01,  5.88654399e-01, -6.88186109e-01,
       -3.25743496e-01, -2.39707753e-01,  7.80149400e-02, -3.85963395e-02,
       -9.85636950e-01,  2.01991439e-01, -8.84459734e-01, -2.94946358e-02,
        4.17395562e-01,  3.98282737e-01,  2.68196434e-01,  5.29761434e-01,
       -7.28140950e-01, -3.61297846e-01,  8.46634626e-01, -1.09496880e-02,
       -6.94804907e-01, -7.19533384e-01,  2.83848077e-01, -1.26034901e-01,
        3.57003003e-01, -

# 유저 대화내용 인코딩

In [ ]:
# 위에서 확인해보았으니 질문에 해당하는 Q열의 전체를 map 함수와 lambda 함수를 사용하여 인코딩해준다.
df['embedding'] = pd.Series([[]] * len(df)) # dummy화
df['embedding'] = df['Q'].map(lambda x: list(model.encode(x)))  # 한 문장씩 인코딩하여 데이터프레임의 새로운 열 embedding에 넣어준다
df.head()

,Q,A,label,embedding
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2,"[-0.4589523, -0.16547582, -1.1699948, -0.52358..."
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1,"[-0.33981282, -0.34263602, -0.1950675, 0.27718..."
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1,"[-0.29754183, -0.046723, 0.31551895, -0.247492..."
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2,"[-0.46993124, 0.12696414, -0.34500104, 0.03754..."
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2,"[0.29756895, -0.15757246, -0.43637148, -0.2524..."


In [ ]:
df.to_csv('wellness_dataset_final.csv', index=False) # 전처리한 데이터 저장 -> 다운받아서 streamlit에서 사용

#간단한 챗봇 코랩에서 동작해보기

#챗봇 테스트

In [ ]:
# 사용자의 질문을 입력받으면 인코딩 해준다
text = '요즘 머리가 아프고 너무 힘들어'
embedding = model.encode(text)

In [ ]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())
df.head()
# 미리 임베딩한 데이터셋(전처리된 데이터)에서 임베딩된 벡터와 사용자 질문의 임베딩된 값을 
# map 함수와 코사인 유사도를 구하여 데이터프레임의 distance열에 저장한다.

,Q,A,label,embedding,distance
0,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,2,"[-0.4589523, -0.16547582, -1.1699948, -0.52358...",0.216589
1,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,1,"[-0.33981282, -0.34263602, -0.1950675, 0.27718...",0.570122
2,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,1,"[-0.29754183, -0.046723, 0.31551895, -0.247492...",0.432349
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,2,"[-0.46993124, 0.12696414, -0.34500104, 0.03754...",0.228684
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,2,"[0.29756895, -0.15757246, -0.43637148, -0.2524...",0.213341


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57832 entries, 0 to 57831
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Q          57832 non-null  object 
 1   A          57832 non-null  object 
 2   label      57832 non-null  int64  
 3   embedding  57832 non-null  object 
 4   distance   57832 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.7+ MB


In [ ]:
# 유사도가 저장되어 있는 distance열에서 코사인 유사도가 가장 max값을 출력
answer = df.loc[df['distance'].idxmax()]

print('유사한 질문', answer['Q'])
print('챗봇 답변', answer['A'])
print('유사도', answer['distance'])

유사한 질문 요즘에 머리가 너무 아파서 걱정이야.
챗봇 답변 요즘에 머리가 너무 아파서 걱정이시군요.무슨 일 있으신가요?
유사도 0.9255013465881348


# 데이터를 하나를 사용했을 때와 병합했을 때 같은 질문에 대한 챗봇의 답변

In [ ]:
## 질문 : '요즘 머리가 아프고 너무 힘들어'
# 데이터합치기 전 답변
# 으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
# 데이터합친 후 답변
# 요즘에 머리가 너무 아파서 걱정이시군요.무슨 일 있으신가요?